# Arc Wall Scanner - Enhanced Training on Google Colab
## Dual Model Training: Specialized + General Detection

**Instructions:**
1. Upload `yolo_data.zip` in Cell 2
2. Run all cells in order
3. Training takes ~2-3 hours on GPU
4. Download both models at the end

In [ ]:
# Cell 1: Install dependencies
!pip install ultralytics opencv-python google-cloud-storage -q
print("✓ Dependencies installed")

In [ ]:
# Cell 2: Upload yolo_data.zip
from google.colab import files
print("Please upload yolo_data.zip...")
uploaded = files.upload()
print("✓ Data uploaded")

In [ ]:
# Cell 3: Extract and verify data
!unzip -q yolo_data.zip
!ls yolo_data/

# Fix data.yaml for Colab environment
import os
data_yaml_content = f"""
train: {os.path.abspath('yolo_data/images/train')}
val: {os.path.abspath('yolo_data/images/train')}

nc: 5
names: ['mirror', 'thermostat', 'vent', 'wall_socket', 'window_box_(for_plants)']
"""

with open('yolo_data/data.yaml', 'w') as f:
    f.write(data_yaml_content)

print(f"Updated data.yaml with absolute paths:")
with open('yolo_data/data.yaml', 'r') as f:
    print(f.read())

!echo "Training images:"
!ls yolo_data/images/train/ | wc -l
!echo "Label files:"
!ls yolo_data/labels/train/ | wc -l
print("✓ Data extracted and verified")
print("✓ data.yaml fixed for Colab environment")

In [ ]:
# Cell 4: Train specialized wall elements model
from ultralytics import YOLO
import torch

print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# Initialize specialized model (YOLOv8m - best balance of speed/accuracy)
wall_model = YOLO('yolov8m.pt')

# Train with optimized parameters
results = wall_model.train(
    data='yolo_data/data.yaml',
    epochs=50,
    batch=32,
    imgsz=640,
    device=0,
    project='runs/detect',
    name='wall_elements_specialized',
    patience=15,
    save=True,
    plots=True,
    # Optimized hyperparameters
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    # Data augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    mosaic=1.0,
    mixup=0.1
)

print("\n✓ Specialized wall elements training completed!")
print(f"Best model: runs/detect/wall_elements_specialized/weights/best.pt")

In [ ]:
# Cell 5: Prepare general detection model
# YOLOv8m is already downloaded and provides good balance of speed/accuracy
print("✓ General detection model (YOLOv8m) ready")
print("This model can detect 80 different object classes")

In [ ]:
# Cell 6: Test both models
import cv2
import matplotlib.pyplot as plt

# Test on a sample image
test_image = "yolo_data/images/train/" + !ls yolo_data/images/train/ | head -1
test_image = test_image[0]

print(f"Testing on: {test_image}")

# Test specialized model
wall_results = wall_model(test_image)
wall_results[0].show()
print(f"Wall elements detected: {len(wall_results[0].boxes) if wall_results[0].boxes else 0}")

# Test general model
general_model = YOLO('yolov8m.pt')
general_results = general_model(test_image)
general_results[0].show()
print(f"General objects detected: {len(general_results[0].boxes) if general_results[0].boxes else 0}")

In [ ]:
# Cell 7: Download trained models and results
from google.colab import files

# Download specialized wall model
files.download('runs/detect/wall_elements_specialized/weights/best.pt')

# YOLOv8m is already available as the general model

# Download training results
files.download('runs/detect/wall_elements_specialized/results.png')
files.download('runs/detect/wall_elements_specialized/confusion_matrix.png')

print("\n🎉 All files downloaded!")
print("\nYou now have:")
print("1. best.pt - Your specialized wall elements model")
print("2. yolov8m.pt - General object detection model (already available)")
print("3. Training results and metrics")
print("\nUse both models together for comprehensive detection!")